In [2]:
import folium
import json
import pandas as pd
import psycopg2

from folium.plugins import MarkerCluster

In [3]:
# Taken from https://github.com/python-visualization/folium/issues/416
# with thanks and recognition to https://github.com/ruoyu0088
# Modified to add a popup
class MarkerClusterScript(MarkerCluster):
    def __init__(self, data, callback, popup=None):
        from jinja2 import Template
        super(MarkerClusterScript, self).__init__([])
        self._name = 'Script'
        self._data = data
        self._popup = popup
        if callable(callback):
            from flexx.pyscript import py2js
            self._callback = py2js(callback, new_name="callback")
        else:
            self._callback = "var callback = {};".format(_callback)

        self._template = Template(u"""
            {% macro script(this, kwargs) %}
            (function(){
                var data = {{this._data}};
                var map = {{this._parent.get_name()}};
                var cluster = L.markerClusterGroup();
                {{this._callback}}

                for (var i = 0; i < data.length; i++) {
                    var row = data[i];
                    var marker = callback(row, popup='names');
                    marker.addTo(cluster);
                }

                cluster.addTo(map);
            })();
            {% endmacro %}
                        """)

In [4]:
def create_marker(row, popup=None):
    """Returns a L.marker object"""
    icon = L.AwesomeMarkers.icon({markerColor: row.color})    
    marker = L.marker(L.LatLng(row.lat, row.lng))
    marker.setIcon(icon)
    if popup:
        marker.bindPopup(row[popup])
    return marker

In [5]:
sql_str = """
    SELECT organisations.name, 
        organisations.id,
        organisations.addresses,
        orgs_latlng.lat,
        orgs_latlng.lng
    FROM gtr.organisations
    LEFT JOIN gtr.orgs_latlng
    ON gtr.organisations.id = gtr.orgs_latlng.id
"""

# Read in config file with DB params
with open('../scripts/config.json') as f:
    conf = json.load(f)
    
# Define a connection string
conn_string = 'host={} dbname={} user={} password={}'.format(conf.get('host'),
                                                             conf.get('database'),
                                                             conf.get('user'),
                                                             conf.get('passw'))

# Create a connection object
conn = psycopg2.connect(conn_string)

# Dataframe from SQL data
df = pd.read_sql(sql_str, conn)

In [6]:
df.head()

,name,id,addresses,lat,lng
0,A-Tech Fabrications Limited,BAB46AB6-90FC-4145-B885-0153BEA15A73,"{'address': [{'city': 'Newton Aycliffe', 'coun...",54.604095,-1.574308
1,Life Sciences,BB979A0C-0F8D-4B3F-A8D0-06A5F0D45FB0,{'address': []},NaN,NaN
2,Zurich Insurance plc,BCD9069A-6739-4039-86DB-042CB13254CB,"{'address': [{'postCode': 'PO15 7JZ', 'line3':...",50.881574,-1.242902
3,Limagrain UK Ltd,B69FACF5-5AB9-4D97-88CD-0A38C32935FF,"{'address': [{'postCode': 'LN7 6DT', 'country'...",53.482100,-0.248484
4,China Earthquake Administration,B76FE143-1E07-40B0-8932-028207296A78,"{'address': [{'country': 'China', 'region': 'O...",48.257585,124.779230


In [7]:
# nan values throw the JS script
lat = df.lat[df.lat.notnull()].values
lng = df.lng[df.lng.notnull()].values

# Popups with name strings
popups = df.name[df.lng.notnull()].values

# Latitude and longitude dataframe with no nan values
locations = [list(a) for a in zip(lat, lng, popups)]
df_locations = pd.DataFrame(locations, columns=['lat', 'lng', 'names'])

In [8]:
fig = folium.element.Figure()
map_orgs = folium.Map(location=[20, 50], zoom_start=1)
MarkerClusterScript(df_locations.to_json(orient="records"), callback=create_marker).add_to(map_orgs)
map_orgs.add_to(fig)
